In [1]:
import numpy as np
import pandas as pd
import polars as pl
# from sklearn.decomposition import NMF as sklearn_nmf
import sys
import os
import subprocess

sys.path.append('/central/groups/mthomson/jboktor/spatial_genomics/osNMF/osNMF')
from knn_impute import kNN_imputation
from morans_index import MoranIforPPs
from other_functions import show_graph_with_labels, filter_genes, weighted_correlation
from sklearn_nmf import sklearn_nmf
from stability import findcorrelation, HungarianError, amariMaxError, instability

In [2]:
# Read the CSV file using polars
filtered_data_polars = pl.read_csv(
    '../data/interim/seurat_data_2024-06-15.csv',
    has_header=True,
    ignore_errors=True
)

# Rename the first column to 'rownames'
first_col_name = filtered_data_polars.columns[0]
filtered_data_polars = filtered_data_polars.rename({first_col_name: "rownames"})

# Convert to pandas DataFrame
filtered_data = filtered_data_polars.to_pandas()

# Set 'rownames' as the index
filtered_data.set_index('rownames', inplace=True)

In [3]:
# filtered_data = pd.read_csv('../data/interim/seurat_data_2024-06-15.csv', index_col=0)
# filtered_data2 = pd.read_csv('../data/interim/2024-06-14_roi1_run1_seurat_data.csv', index_col=0)
# filtered_data2.head()

filtered_data.head()

,roi1_run1_1,roi1_run1_2,roi1_run1_3,roi1_run1_4,roi1_run1_5,roi1_run1_6,roi1_run1_7,roi1_run1_8,roi1_run1_9,roi1_run1_10,...,roi4_run3_74410,roi4_run3_74411,roi4_run3_74412,roi4_run3_74413,roi4_run3_74414,roi4_run3_74415,roi4_run3_74416,roi4_run3_74417,roi4_run3_74418,roi4_run3_74419
rownames,,,,,,,,,,,,,,,,,,,,,
CAMK2A,1.386294,0.693147,0.693147,0.0,0.693147,0.0,1.098612,0.693147,1.386294,0.693147,...,0.693147,3.367296,1.386294,2.564949,0.693147,1.098612,1.098612,1.386294,1.94591,0.000000
SYP,2.708050,0.693147,0.000000,0.0,0.000000,0.0,0.693147,0.693147,0.693147,0.693147,...,0.000000,0.000000,2.564949,0.000000,0.693147,1.791759,1.791759,1.386294,0.00000,1.609438
FUS,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
KIF5A,1.609438,1.098612,0.693147,0.0,0.693147,0.0,1.098612,1.098612,1.386294,1.098612,...,0.693147,1.386294,2.079442,2.484907,0.693147,2.302585,2.639057,1.945910,1.94591,1.098612
HPCA,0.693147,0.693147,0.000000,0.0,0.000000,0.0,0.693147,0.693147,0.693147,0.693147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.00000,0.000000


# Stability Analysis

In [16]:
# Define the parameters
folder_path = '/central/groups/mthomson/jboktor/spatial_genomics/jess_2024-01-23/data/interim/osNMF/instability_runs'
K1=2        # sets lower bound (including itself) of PPs
K2=3       # sets upper bound (including itself) of PPs 
B=100        # sets number of replicates
shape_flat = filtered_data.shape[1]
numPatterns = np.arange(K1, K2+1)


# Loop through the K value pairs and submit the SLURM job
for i in numPatterns:
    name_DG="DG_PP"
    output_file = f"results_K={i}.pkl"
    cmd = [
        "sbatch",
        "/central/groups/mthomson/jboktor/spatial_genomics/jess_2024-01-23/notebooks/shell_scripts/run_instability.sh",
        folder_path,
        str(i),
        str(i),
        str(B),
        str(shape_flat),
        str(name_DG),
        str(output_file)
    ]
    # print(cmd)
    subprocess.run(cmd)

Submitted batch job 42152191
Submitted batch job 42152192
Submitted batch job 42152193
Submitted batch job 42152194
Submitted batch job 42152195
Submitted batch job 42152196
Submitted batch job 42152197
Submitted batch job 42152198
Submitted batch job 42152199
Submitted batch job 42152200
Submitted batch job 42152201
Submitted batch job 42152202
Submitted batch job 42152203
Submitted batch job 42152204
Submitted batch job 42152205
Submitted batch job 42152206
Submitted batch job 42152207
Submitted batch job 42152208
Submitted batch job 42152209
Submitted batch job 42152210
Submitted batch job 42152211
Submitted batch job 42152212
Submitted batch job 42152213
Submitted batch job 42152214
Submitted batch job 42152215
Submitted batch job 42152216
Submitted batch job 42152217
Submitted batch job 42152218
Submitted batch job 42152219
Submitted batch job 42152220
Submitted batch job 42152221
Submitted batch job 42152222
Submitted batch job 42152223
Submitted batch job 42152224
Submitted batc

In [ ]:
# Set key variables
K1=5        # sets lower bound (including itself) of PPs
K2=7       # sets upper bound (including itself) of PPs 
B=100        # sets number of replicates
start=0        # start number for naming replicates
wkdir = os.getcwd()
# folder_path = os.path.join(wkdir, '../data/interim/osNMF/instability_runs')
folder_path = '../data/interim/osNMF/instability_runs'

# Set intermediate variables
numPatterns = np.arange(K1, K2+1)
name_DG="DG_PP"
shape_flat = filtered_data.shape[1]

K_values = [(10, 20), (30, 40)]  # Add more pairs as needed


for i, (K1, K2) in enumerate(K_values):
    print(f"Running for K1={K1}, K2={K2}")


In [ ]:

# Run DG _num_set times to generate _num_set PPs
for k in range(len(numPatterns)):
    K = numPatterns[k]
    # set new directory for K
    path = folder_path + '/K=' + str(K)
    os.makedirs(path, exist_ok=True)

    # prep for runs
    print("Working on K = {}...".format(K))
    
    for b in range(B):
        print("Working on replicate {}...".format(b))
        # Bootstrap data and use same bootstrap input for osNMF
        np.random.seed(b)
        X = np.maximum(filtered_data,0)
        n_samples = X.shape[0]
        random_indices = np.random.choice(n_samples, n_samples, replace=True)
        bootstrap_X = X.iloc[random_indices]
        # bootstrap_X = X[np.random.choice(
        #     n_samples,
        #     n_samples,
        #     replace=True)]

        # run and save PP replicates
        nmf = sklearn_nmf(n_components=K,
                          l1_ratio=1,
                          alpha = 0,
                          max_iter=1000,
                          random_state=1) # random state=1 for reproducibility
        nmf.fit(bootstrap_X) # fit model with bootstrapped data
        PPs_tmp = nmf.components_
        np.savez(path + '/DG_PP_' + str(b+start),PPs_tmp=PPs_tmp)

In [ ]:
# Run instability
print("Run instability for osNMF:")
instability_DG, instability_std_DG, distMat_DG = instability(_folder_path=folder_path,
                                                             _k1=K1,
                                                             _k2=K2,
                                                             _numReplicates=B,
                                                             _n_features=shape_flat,
                                                             _name=name_DG)

print("\n DG instability values:", instability_DG)

In [ ]:
instability_DG

In [ ]:
distMat_DG

In [ ]:
# n_samples
random_indices = np.random.choice(n_samples, n_samples, replace=True)
X_sampled = X.iloc[random_indices]
# X_sampled

In [ ]:
X.shape

In [ ]:
nmf = sklearn_nmf(n_components=3, l1_ratio=1, alpha=0, random_state=1, max_iter=1000)
nmf.fit(np.maximum(filtered_data, 0))
PPs = nmf.components_
coeffs = nmf.transform(np.maximum(filtered_data, 0))